In [1]:
import sys
print(sys.executable)  # This should point to your virtual environment

import pkg_resources
print([p.key for p in pkg_resources.working_set if 'langchain' in p.key.lower()])

/Users/samuelalamsjah/Documents/Projects/EY-Ai-Model/venv/bin/python
['langchain', 'langchain-community', 'langchain-core', 'langchain-openai', 'langchain-text-splitters']


/var/folders/zr/r83vtpz93635tbwyvw749wn40000gn/T/ipykernel_55938/1930377967.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [15]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict, Tuple

from dotenv import load_dotenv
import os

In [16]:
load_dotenv()

# Get API key from environment variables
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

In [17]:
# Define the state structure
class AgentState(TypedDict):
    messages: List[Dict]
    current_step: int
    
# Initialize the LLM and search tool
llm = ChatOpenAI(temperature=0) # temperature 0 gives more accuracy
search = GoogleSerperAPIWrapper()


In [26]:
async def async_search(query: str) -> str:
    # returns a dictionary
    search_results = search.results(query)

    # Extract organic search results
    organic_results = search_results.get('organic', [])

    # Format the results
    formatted_results = []
    for result in organic_results[:3]:  # Get top 3 results
        title = result.get('title', '')
        link = result.get('link', '')
        snippet = result.get('snippet', '')
        formatted_results.append(f"Title: {title}\nLink: {link}\nSummary: {snippet}\n")
    
    print("\n".join(formatted_results))
    
    return "\n".join(formatted_results)

In [27]:
# Create tools list
tools = [
    Tool(
        name="web_search",
        description="Searches google for information using Serper API",
        func=async_search,
        coroutine=async_search  # This is important for async tools
    )
]

In [28]:
# Define agent functions
def should_continue(state: AgentState) -> bool:
    """Determine if the agent should continue or end."""
    return state["current_step"] < 1

async def search_and_analyze(state: AgentState) -> AgentState:
    """Search the web and analyze results asynchronously."""
    messages = state["messages"]
    last_message = messages[-1]["content"]
    
    # Perform async search
    search_result = await async_search(last_message)
    
    # Analyze results with LLM
    response = await llm.ainvoke([
        HumanMessage(content=f"""
        Based on this search result: {search_result}
        Please provide a clear and concise answer to: {last_message}
        """)
    ])
    
    messages.append({"role": "assistant", "content": response.content})
    state["current_step"] += 1
    return state

In [29]:
# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("search_and_analyze", search_and_analyze)

# Add the starting edge from START to our first node
workflow.set_entry_point("search_and_analyze")

# Add edges
workflow.add_conditional_edges(
    "search_and_analyze",
    should_continue,
    {
        True: "search_and_analyze",
        False: END
    }
)

# Compile the graph
chain = workflow.compile()

In [30]:
# Example usage
initial_state = {
    "messages": [{"role": "user", "content": "Give me an overview of the oil Industry condition in Indonesia the past 3 years"}],
    "current_step": 0
}

# Run the agent
result = await chain.ainvoke(initial_state)

Title: Indonesia - Countries & Regions - IEA
Link: https://www.iea.org/countries/indonesia/oil
Summary: Indonesia's imports of oil have rapidly increased in recent years. This resource-rich nation is the world's fourth-largest producer of coal and Southeast ...

Title: Oil production in Indonesia 2012-2022 - Statista
Link: https://www.statista.com/statistics/609047/oil-production-in-indonesia/
Summary: Indonesia's oil production has been declining in the last decades largely due to its aging oil fields and constantly changing regulations that ...

Title: Energy Resource Guide - Indonesia - Oil and Gas
Link: https://www.trade.gov/energy-resource-guide-indonesia-oil-and-gas
Summary: In 2019, the country's oil production was 746,800 barrels per day (bpd), lower than 2017's production of 803,000 bpd. 2018 production decreased again with 778, ...



In [23]:
# Print results
for message in result["messages"]:
    if message["role"] == "assistant":
        print(f"AI: {message['content']}\n")

AI: Some of the latest developments in AI include the shift towards more pragmatic approaches, advancements in generative AI beyond chatbots, the emergence of AI agents as the next frontier, the development of new technologies such as GPT-4, GANs, XAI, and NLP, and the exploration of AI trends in robotics and machine learning for various applications. Additionally, there have been significant investments in AI leading to transformative advances impacting everyday lives, such as mapping technologies, voice-assisted smartphones, and handwriting recognition.

AI: Some of the latest developments in AI include pragmatic approaches, advancements in generative AI beyond chatbots, the emergence of AI agents as the next frontier, the development of new technologies such as GPT-4, GANs, XAI, and NLP, and the exploration of AI trends in robotics and machine learning for various applications. Additionally, significant investments in AI have led to transformative advances impacting everyday lives, 